# Calculate Pacific Overturning streamfunction

- the MPI low res, I would probably have to regrid before doing the streamfunction? Otherwise its not really meaningful?

- is the combination with different variables the problem

- but the problem seems to exist in intake-esm already...maybe its time to ditch that...

## TODO: 

- I need to redo the regressions for vo in the correct folder!
- Preserve attrs

In [4]:
%load_ext autoreload
%autoreload 2
import warnings
import pathlib
import cf_xarray
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from xarrayutils.filtering import filter_1D
from xarrayutils.file_handling import write, maybe_create_folder
from dask.diagnostics import ProgressBar

from xgcm import Grid

import dask
# dask.config.set(**{'array.slicing.split_large_chunks': True, "array.chunk-size": "32GB",})
from aguadv_omz_busecke_2021.cmip6_stash import cmip6_dataset_id
# from aguadv_omz_busecke_2021.utils import reg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# from euc_dynamics.utils import master_load, force_datasets_on_same_grid, mask_pacific, combine_grid_labels, o2_models
from euc_dynamics.cmip6_preprocessing_stash import regrid_horizontal
from euc_dynamics.xarrayutil_stash import map_nested_dict

In [6]:
from aguadv_omz_busecke_2021.omz_tools import mask_pacific
from aguadv_omz_busecke_2021.cmip6_stash import (
    load_single_datasets,
    match_and_detrend,
    nested_dataset_wrapper,
    parse_metrics,
)
from aguadv_omz_busecke_2021.utils import stci, read_files

In [7]:
# foldername = "density_remapping_obs"
# ofolder = maybe_create_folder(f"../../data/processed/{foldername}")

In [8]:
overwrite = True
odir = maybe_create_folder(pathlib.Path('../../data/processed/pacific_meridional_overturning'))
odir_filtered = maybe_create_folder(pathlib.Path('../../data/processed/pacific_meridional_overturning/filtered'))

/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/xarrayutils/file_handling.py:118: UserWarning: Folder ../../data/processed/pacific_meridional_overturning does already exist.
  warnings.warn(f"Folder {path} does already exist.", UserWarning)
/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/xarrayutils/file_handling.py:118: UserWarning: Folder ../../data/processed/pacific_meridional_overturning/filtered does already exist.
  warnings.warn(f"Folder {path} does already exist.", UserWarning)


## Load detrended datasets (before activating the dask scheduler, since that messes with this somehow)

In [9]:
import intake
from aguadv_omz_busecke_2021.utils import cmip6_collection
col = intake.open_esm_datastore(cmip6_collection(zarr=True))

In [10]:
ddict = load_single_datasets(
    col,
    source_id=[
        "UKESM1-0-LL", # area not available for all members
        "MRI-ESM2-0", # 
        "NorESM2-LM",
        "NorESM2-MM",
        "GFDL-ESM4",
        "GFDL-CM4",
        "MIROC-ES2L", # done
        "CNRM-ESM2-1", # done bunch of members have missing thetao
        "CanESM5", # done
        "CanESM5-CanOE", # done
        "MPI-ESM1-2-HR", # done
        "ACCESS-ESM1-5", # done
        "IPSL-CM6A-LR", # done
        "NorESM1-F", #meh, dont need it
        "NorCPM1", #meh, dont need it
        "MPI-ESM-1-2-HAM", # done
#         "MPI-ESM1-2-LR", # done
    ],
    grid_label=["gr", "gn"],
    experiment_id=["historical", "ssp585"],
    table_id=["Omon"],
    variable_id=["vo"],
)

Dataframe size before picking latest version: 227
Getting latest version...

Dataframe size after picking latest version: 227

Done....


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.version.zstore'
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found


In [11]:
list(ddict.keys())

['CMIP.NCC.NorCPM1.historical.r5i1p1f1.Omon.vo.gr',
 'CMIP.NCC.NorCPM1.historical.r16i1p1f1.Omon.vo.gr',
 'ScenarioMIP.NOAA-GFDL.GFDL-ESM4.ssp585.r1i1p1f1.Omon.vo.gn',
 'CMIP.NCC.NorCPM1.historical.r26i1p1f1.Omon.vo.gr',
 'CMIP.NCC.NorCPM1.historical.r8i1p1f1.Omon.vo.gr',
 'CMIP.NCC.NorCPM1.historical.r3i1p1f1.Omon.vo.gr',
 'CMIP.NCC.NorCPM1.historical.r22i1p1f1.Omon.vo.gr',
 'CMIP.NCC.NorCPM1.historical.r25i1p1f1.Omon.vo.gr',
 'ScenarioMIP.CCCma.CanESM5.ssp585.r4i1p1f1.Omon.vo.gn',
 'ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp585.r4i1p1f1.Omon.vo.gn',
 'ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp585.r6i1p1f1.Omon.vo.gn',
 'ScenarioMIP.NCC.NorESM2-LM.ssp585.r1i1p1f1.Omon.vo.gn',
 'ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp585.r2i1p1f1.Omon.vo.gn',
 'ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.r1i1p1f2.Omon.vo.gn',
 'ScenarioMIP.NCC.NorESM2-MM.ssp585.r1i1p1f1.Omon.vo.gr',
 'ScenarioMIP.CCCma.CanESM5.ssp585.r9i1p2f1.Omon.vo.gn',
 'ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp585.r14i1p1f1.Omon.vo.gn',
 'ScenarioMIP.CNRM-CERFACS.CNRM-

In [12]:
#temporary fix
def parse_member_id(name, ds):
    member_id = name.split('.')[-4]
    if 'member_id' not in ds.dims:
        print(f"no member id in dims for {name}")
    else:
        print(f"Found {ds.member_id.data} for {name}")
        
    ds = ds.assign_coords(member_id=xr.DataArray([member_id], dims=['member_id']))
    return ds

ddict_preprocessed = {k:parse_member_id(k,ds) for k, ds in ddict.items()}

Found [0] for CMIP.NCC.NorCPM1.historical.r5i1p1f1.Omon.vo.gr
Found [0] for CMIP.NCC.NorCPM1.historical.r16i1p1f1.Omon.vo.gr
Found ['r1i1p1f1'] for ScenarioMIP.NOAA-GFDL.GFDL-ESM4.ssp585.r1i1p1f1.Omon.vo.gn
Found [0] for CMIP.NCC.NorCPM1.historical.r26i1p1f1.Omon.vo.gr
Found [0] for CMIP.NCC.NorCPM1.historical.r8i1p1f1.Omon.vo.gr
Found [0] for CMIP.NCC.NorCPM1.historical.r3i1p1f1.Omon.vo.gr
Found [0] for CMIP.NCC.NorCPM1.historical.r22i1p1f1.Omon.vo.gr
Found [0] for CMIP.NCC.NorCPM1.historical.r25i1p1f1.Omon.vo.gr
Found [0] for ScenarioMIP.CCCma.CanESM5.ssp585.r4i1p1f1.Omon.vo.gn
Found [0] for ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp585.r4i1p1f1.Omon.vo.gn
Found [0] for ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp585.r6i1p1f1.Omon.vo.gn
Found [0] for ScenarioMIP.NCC.NorESM2-LM.ssp585.r1i1p1f1.Omon.vo.gn
Found [0] for ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp585.r2i1p1f1.Omon.vo.gn
Found [0] for ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.r1i1p1f2.Omon.vo.gn
Found [0] for ScenarioMIP.NCC.NorESM2-MM.ssp585.r1i1p1f1.Omo

In [13]:
# are there any gr data for mpi-lr? nope
# col.search(source_id="MPI-ESM1-2-LR", grid_label='gr')

In [14]:
# Load all trends into dict

import pathlib
from fastprogress.fastprogress import progress_bar

trendfolder = pathlib.Path('../../data/processed/linear_regression_time_zarr_multimember/')
trend_models = np.unique([ds.attrs['source_id'] for ds in ddict.values()])
flist = []
for tm in trend_models:
    flist = flist + list(trendfolder.glob(f'*{tm}*_trend.nc'))

total = len(flist)
progress = progress_bar(range(total))

trend_dict = {}
for i,path in enumerate(flist):
    key = path.stem
    ds = xr.open_mfdataset([path])
    # write the filename in the dataset
    ds.attrs.update({'filepath':str(path)})
    # exclude all nan slopes (why are these there in the first place?)
    if not np.isnan(ds.slope).all():
        trend_dict[key] = ds
    else:
        print(f"found all nan regression data for {path}")
    progress.update(i)
progress.update(total)

found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r22i1p1f1_so_trend.nc
found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r23i1p1f1_o2_trend.nc
found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r25i1p1f1_so_trend.nc
found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r21i1p1f1_o2_trend.nc
found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r24i1p1f1_so_trend.nc
found all nan regression data for ../../data/processed/linear_regression_time_zarr_multimember/CMIP_CCCma_CanESM5_historical_Omon_gn_v20190429_r22i1p1f1_o2_trend.nc
found all 

In [26]:
list(trend_dict.keys())

['CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r10i1p1f1_so_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r4i1p1f1_so_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r2i1p1f1_agessc_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r2i1p1f1_so_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r5i1p1f1_o2_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r9i1p1f1_agessc_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r4i1p1f1_agessc_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r5i1p1f1_thetao_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r6i1p1f1_o2_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r8i1p1f1_so_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r1i1p1f1_thetao_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r2i1p1f1_o2_trend',
 'CMIP_CSIRO_ACCESS-ESM1-5_historical_Omon_gn_v20200529_r6i1p1f1_so

In [16]:
ddict_detrended = match_and_detrend(
    ddict_preprocessed,
    trend_dict,
    verbose=True,
    pass_variables=["areacello", "thkcello", "mlotst"],
)

/projects/GEOCLIM/LRGROUP/jbusecke/projects/aguadv_omz_busecke_2021/aguadv_omz_busecke_2021/cmip6_stash.py:261: UserWarning: No match found for ['NorCPM1', 'Omon', 'gr', 'r5i1p1f1', 'vo'].
  warnings.warn(f"No match found for {match_elements}.")
/projects/GEOCLIM/LRGROUP/jbusecke/projects/aguadv_omz_busecke_2021/aguadv_omz_busecke_2021/cmip6_stash.py:261: UserWarning: No match found for ['NorCPM1', 'Omon', 'gr', 'r16i1p1f1', 'vo'].
  warnings.warn(f"No match found for {match_elements}.")
/projects/GEOCLIM/LRGROUP/jbusecke/projects/aguadv_omz_busecke_2021/aguadv_omz_busecke_2021/cmip6_stash.py:261: UserWarning: No match found for ['GFDL-ESM4', 'Omon', 'gn', 'r1i1p1f1', 'vo'].
  warnings.warn(f"No match found for {match_elements}.")
/projects/GEOCLIM/LRGROUP/jbusecke/projects/aguadv_omz_busecke_2021/aguadv_omz_busecke_2021/cmip6_stash.py:261: UserWarning: No match found for ['NorCPM1', 'Omon', 'gr', 'r26i1p1f1', 'vo'].
  warnings.warn(f"No match found for {match_elements}.")
/projects/GE

In [17]:
list(ddict_detrended.keys())

['ScenarioMIP.CCCma.CanESM5-CanOE.ssp585.r2i1p2f1.Omon.vo.gn',
 'CMIP.CCCma.CanESM5-CanOE.historical.r2i1p2f1.Omon.vo.gn',
 'CMIP.CCCma.CanESM5-CanOE.historical.r1i1p2f1.Omon.vo.gn',
 'ScenarioMIP.CCCma.CanESM5-CanOE.ssp585.r1i1p2f1.Omon.vo.gn',
 'ScenarioMIP.CCCma.CanESM5-CanOE.ssp585.r3i1p2f1.Omon.vo.gn',
 'CMIP.CCCma.CanESM5-CanOE.historical.r3i1p2f1.Omon.vo.gn']

In [18]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    ddict_nested = nested_dataset_wrapper(ddict_detrended)

Could not find reference dataset for variables ['vo'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for variables ['vo'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for variables ['vo'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for variables ['vo'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for variables ['vo'] and grid_label gn
No datasets to merge in `combine_grid_labels`.
Could not find reference dataset for variables ['vo'] and grid_label gn
No datasets to merge in `combine_grid_labels`.


In [20]:
# ddict_nested['CMIP.MPI-M.MPI-ESM1-2-LR.historical.r10i1p1f1.Omon']

In [21]:
# Extra step, regrid to regular lon/lat grid
# (for now only for MPI-LR, as far as I can tell this is the only one with
# a non regular grid in the tropics)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    ddict_nested_regridded = {
        k: regrid_horizontal(
            ds.drop(["lon_verticies", "lat_verticies"]), 0.25, 0.25
        ).assign_coords(lev_bounds=ds.lev_bounds, member_id=ds.member_id)
        for k, ds in ddict_nested.items()
    }

AttributeError: 'NoneType' object has no attribute 'drop'

In [17]:
# ddict_nested

In [18]:
ddict_nested_metrics = parse_metrics(ddict_nested_regridded)

In [19]:
# ddict_nested_metrics

## Notes
- the combined fields might be a bit dicey, when I interpolate them from gr to gn...maybe leave those out? At least clearly mark them?

- these grids dont have an outer z coordinate. That should be fixed...

In [ ]:
for name, ds in ddict_nested_metrics.items():

    ds_id = cmip6_dataset_id(ds)

    print(f"Processing {ds_id}")

    # This needs to go to cmip6_preprocessing
    lev_outer = cf_xarray.bounds_to_vertices(ds.lev_bounds, bounds_dim='bnds')
    ds = ds.assign_coords(lev_outer = lev_outer.data)
    ds.lev_outer.attrs.update({'axis':'Z', 'c_grid_axis_shift':0.5})
    grid = Grid(ds, metrics={'X':['dx_t'], 'Y':'dy_t', 'Z':'dz_t'}) # I should be able to parse those...

    ds_pac = mask_pacific(ds)
    ds_pac

    v_int = grid.integrate(ds.vo, 'X')
    psi = grid.cumint(v_int, 'Z', boundary='fill').chunk({'y':-1, 'lev_outer':-1})

    # convert to dataset
    ds_write = psi.to_dataset(name='psi')
    ds_write = ds_write.assign_coords(member_id=ds.member_id)
    
    # restor attributes from original dataset
    
    ds_write.attrs.update({k:v for k,v in ds.attrs.items() if k not in ['intake_esm_varname']})
    print(ds_write)
    
    filename = f"pacific_meridional_overturning_{ds_id}_{str(ds.member_id.data)}.nc"
    print(filename)
    
    with ProgressBar():
        ds_check = write(ds_write, odir.joinpath(filename), force_load=True, serialize_load_over_dim=False, overwrite=overwrite)

    # control plots
    plt.figure(figsize=[15,4])
    plt.subplot(1,2,1)
    ds_check.psi.mean('time').sel(lev_outer=slice(0,750)).plot(robust=True, x='y', yincrease=False)
    plt.title('')

    plt.subplot(1,2,2)
    ds_check.psi.sel(y=10, lev_outer=70, method='nearest').resample(time='10AS').mean().plot()
    plt.title('')
    plt.suptitle(filename)
    plt.show()

Processing CMIP_MPI-M_MPI-ESM1-2-LR_historical_Omon_gn_none


/scratch/gpfs/jbusecke/conda_tigercpu/envs/aguadv_omz_busecke_2021/lib/python3.8/site-packages/xarray/core/indexing.py:1375: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


<xarray.Dataset>
Dimensions:                (lev_outer: 41, member_id: 1, time: 1980, y: 720)
Coordinates:
  * y                      (y) float64 -89.88 -89.62 -89.38 ... 89.62 89.88
  * lev_outer              (lev_outer) float64 0.0 12.0 ... 5.42e+03 6.02e+03
  * time                   (time) object 1850-01-16 12:00:00 ... 2014-12-16 1...
    branch_time_in_parent  (member_id) float64 1.644e+05
    branch_time_in_child   (member_id) float64 0.0
    parent_time_units      (member_id) <U28 'days since 1850-1-1 00:00:00'
    child_time_units       (member_id) <U38 'hours since 1850-01-16 12:00:00....
    parent_variant_label   (member_id) <U8 'r1i1p1f1'
  * member_id              (member_id) <U9 'r10i1p1f1'
Data variables:
    psi                    (y, lev_outer, time) float64 dask.array<chunksize=(720, 41, 3), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7 CMIP-6.2
    activity_id:             CMIP
    branch_method:           standard
    cmor_version:            3.5

## Filter and combine the Psi data

In [ ]:
def psi_filter(ds, plot=True, filter_dist=1):
    """Filter streamfunction fields in latitude to avoid local minima/maxima due to noise later
    filter_dist: float
        Std of the filter kernel to be applied in deg. The actual kernel setup will be inferred from data
    """
    ds_id = cmip6_dataset_id(ds)
    # base the filter window size on the dlat spacing near the equator 
    #(this is pretty ad hoc, but at least it will provide some more comparable results)
    dlat_ref = ds.y.diff('y').sel(y=0, method='nearest').data
    filter_win = int(filter_dist/dlat_ref)
    if filter_win < 1:
        print('reset kernel std to 1')
        filter_win = 1
    ds_filtered = filter_1D(ds, filter_win, 'y')
    print(f"Filtering in latitude with {filter_win} bin kernel, based on minimal dlat distance at equator of ~ {dlat_ref:.2} deg")
    if plot:
        with ProgressBar():
            plt.figure()
            da = ds_filtered.psi.squeeze()
            print(da)
            kwargs = {}
            
            if 'member_id' in da.dims:
                da = da.isel(member_id=slice(0,5))
                kwargs.update(dict(col='member_id', col_wrap=5))
            da.isel(time=slice(0,24)).mean('time').sel(lev_outer=slice(0,500), y=slice(-35,35)).plot(yincrease=False, **kwargs)
            plt.suptitle(ds_id)
            plt.show()
#             plt.figure(figsize=[14,3])
#             da.sel(y=5, lev_outer=50, method='nearest').plot(hue='member_id')
    return ds_filtered

In [ ]:
stc_dict_raw = read_files(
    "../../data/processed/pacific_meridional_overturning/", read_chunks={"time": 6},
)

In [ ]:
stc_dict_raw['MPI-ESM1-2-LR']

In [ ]:
stc_dict = map_nested_dict(stc_dict_raw, psi_filter)

**Note**: Is the read routine actually combining/aligning members? I should probably do that later.

In [ ]:
for model in stc_dict.keys():
    for experiment in stc_dict[model].keys():
        ds_raw = stc_dict[model][experiment]
        ds = (
            ds_raw
            .sel(lev_outer=slice(0, 500), y=slice(-30, 30))
            .resample(time="1AS")
            .mean()
        )
        # for testing
        with ProgressBar():
            ds_reload = write(
                ds,
                odir_filtered.joinpath(f"temp_{model}_{experiment}.nc"),
                force_load = False,
                overwrite=overwrite,
            )
        plt.figure()
        ds_reload.psi.sel(y=5, lev_outer=50, method='nearest').plot(hue='member_id')  
        plt.figure()
        ds_reload.psi.mean('time').plot(col='member_id', col_wrap=4)

In [ ]:
print('Done')

In [ ]:
!conda list